In [37]:
import numpy as np
import collections
import networkx as nx
import matplotlib.pyplot as plt
import random
import glob

def load_ego_nets(input_dir, network_type='gplus'):
    """
    Load ego nets from files in input_dir.
    Returns a dictionary of networkx graphs.
    """
    ego_nets = {}
    for file in glob.glob(input_dir + '/*.edges'):
        name = file.split('/')[-1].split('.')[0]
        if network_type in ['twitter', 'gplus']:
            ego_net = nx.read_edgelist(file, nodetype=str, create_using=nx.DiGraph)
        elif network_type == 'facebook':
            ego_net = nx.read_edgelist(file, nodetype=str, create_using=nx.Graph)

        ego_net.add_node(name)

        for n in ego_net.nodes():
            if n != name:
                ego_net.add_edge(name, n)

        featnames_dir = {}

        with open(input_dir + '/' + name + '.featnames') as f:
            featnames = f.read().splitlines()

        for featname in featnames:
            id, f = featname.split(' ', 1)
            
            if network_type == 'twitter':
                k, v = f, ''
            else:
                k, v = f.split(':', 1)

            if k not in featnames_dir:
                featnames_dir[int(id)] = (k, v)

        feats_dir = {}
        feats = []

        with open(input_dir + '/' + name + '.egofeat') as f:
            egofeat = [name] + f.read().splitlines()[0].split()
            feats.append(egofeat)

        with open(input_dir + '/' + name + '.feat') as f:
            for feat in f.read().splitlines():
                feats.append(feat.split())

        for feat in feats:
            if network_type in ['gplus', 'facebook']:
                temp = {}
                for i, f in enumerate(feat[1:]):
                    if int(f) == 1:
                        temp[featnames_dir[i][0]] = featnames_dir[i][1]

            elif network_type == 'twitter':
                temp = {'hashtags': [], 'follows': []}

                for i, f in enumerate(feat[1:]):
                    if int(f) == 1:
                        if featnames_dir[i][0].startswith('#'):
                            temp['hashtags'].append(featnames_dir[i][0])
                        elif featnames_dir[i][0].startswith('@'):
                            temp['follows'].append(featnames_dir[i][0])

            temp['neighbors'] = list(ego_net.neighbors(feat[0]))

            feats_dir[feat[0]] = temp

        nx.set_node_attributes(ego_net, feats_dir, 'features')

        ego_nets[name] = ego_net

    return ego_nets

def form_communities(G):

    features = nx.get_node_attributes(G, 'features')

    prompt = f"""
    # Task

    Your task is to form communities of people based on their features.

    # Input
    The input is given below after chevrons

    <FEATURES>
    {json.dumps(features, indent=4)}
    </FEATURES>

    # Output
    Your outputs should be formatted as a list of JSON objects, as follows:

    [
        {{
            "name": name of the community,
            "members": list of people in the community,
            "reason" : reason for forming the community
        }}, ...
    ]

    # Notes
    1. The communities should be formed based on the features of the people.
    2. The communities may be overlapping.

    ```json
    """



NodeDataView({'16287561': {'features': {'hashtags': ['#TheApprentice', '#boatrace', '#prometheus'], 'follows': ['@ProfBrianCox', '@Raspberry_Pi', '@TheNextWeb:']}}, '16592928': {'features': {'hashtags': ['#3:', '#LBP', '#LBP2', '#lbp2', '#uncharted2'], 'follows': ['@LittleBigLand', '@LittleBigLand:', '@MediaMolecule', '@Media_Molecule', '@Media_Molecule:', '@PlayStation', '@SonyPlayStation', '@VofEscaflowne', '@jfairbairn', '@mmalex:', '@mrfungfung:', '@sidshuman:']}}, '14691709': {'features': {'hashtags': ['#13'], 'follows': ['@CTZ:']}}, '1344951': {'features': {'hashtags': [], 'follows': ['@wired']}}, '67393327': {'features': {'hashtags': ['#BAFTAGames', '#SoundCloud', '#goodtimes', '#lbp2', '#wi'], 'follows': ['@BAFTAGames:', '@BrianKaris:', '@HAL9000_:', '@Johnee_B', '@LittleBigPlanet', '@Ms_Czarny', '@bengoldacre:', '@coasterkid93', '@djs18', '@frankieboyle:', '@gavinsandie', '@jonnyhopper', '@mediamolecule', '@michyboo', '@mmalex:', '@rexbox', '@thespaff', '@tomkiss']}}, '1586249